In [1]:
import _init_paths
import caffe
import numpy as np
import os.path as osp

In [2]:
caffe.set_random_seed(13397)

In [3]:
print osp.abspath(_init_paths.root_dir)

/home/abhijit/Workspace/RenderAndCompare


In [4]:
net = caffe.Net('SoftmaxWithTemperature_Caffe.prototxt', caffe.TEST)

In [5]:
net.blobs['data'].data[...] = np.array([[0, 200, 10], [0, 10, 200], [200, 200, 200], [-1, 0, 1], [1, 1, 0]])
net.blobs['T'].data[...] = np.array([4.0, 1.0, 0.5, 0.1, 0.01])

In [6]:
output = net.forward()

In [7]:
print output['softmax_prob_fixedT']

[[ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.33333334  0.33333334  0.33333334]
 [ 0.          0.          1.        ]
 [ 0.5         0.5         0.        ]]


In [8]:
print output['softmax_prob2']

[[ 0.14400937  0.28939405  0.25400397  0.3125926 ]
 [ 0.00891814  0.01068074  0.91332608  0.06707503]
 [ 0.00278466  0.00175055  0.8045789   0.19088584]
 [ 0.20243488  0.2364154   0.30601174  0.25513792]
 [ 0.29135174  0.25418067  0.12491894  0.32954866]]


In [9]:
output['softmax_prob2'].sum(axis=1)

array([ 1.        ,  1.        ,  0.99999994,  0.99999994,  1.        ], dtype=float32)

In [10]:
output['softmax_prob_fixedT'].sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

In [18]:
fc1_by_T = net.blobs['fc1'].data / net.blobs['T'].data.reshape(-1,1)
assert np.allclose(fc1_by_T, net.blobs['fc1_by_T'].data)

In [21]:
net.blobs['fc1_by_T'].data

array([[  5.04695326e-02,   7.48380423e-01,   6.17941082e-01,
          8.25491905e-01],
       [ -3.93103063e-02,   1.41044021e-01,   4.58969545e+00,
          1.97841430e+00],
       [  6.00380421e+00,   5.53960991e+00,   1.16699963e+01,
          1.02313528e+01],
       [ -1.44830182e-01,   1.03421025e-02,   2.68375099e-01,
          8.65559652e-02],
       [  1.52571511e+00,   1.38922930e+00,   6.78848922e-01,
          1.64890790e+00]], dtype=float32)

In [22]:
def softmax(x, t=1.0):
    if x.ndim == 1:
        x = x.reshape((1, -1))
    assert x.ndim == 2
    max_x = np.max(x, axis=1).reshape((-1, 1))
    exp_x = np.exp((x - max_x) / t)
    out = exp_x / np.sum(exp_x, axis=1).reshape((-1, 1))
    return out

In [25]:
softmax(net.blobs['fc1_by_T'].data).sum(axis=1)

array([ 1.        ,  1.        ,  0.99999994,  0.99999994,  1.        ], dtype=float32)

In [24]:
output['softmax_prob_fixedT'].sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)